Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Distributed Tensorflow with Horovod
In this tutorial, you will train a word2vec model in TensorFlow using distributed training via [Horovod](https://github.com/uber/horovod).

## Prerequisites
* Understand the [architecture and terms](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture) introduced by Azure Machine Learning (AML)
* Go through the [00.configuration.ipynb](https://github.com/Azure/MachineLearningNotebooks/blob/master/00.configuration.ipynb) notebook to:
    * install the AML SDK
    * create a workspace and its configuration file (`config.json`)
* Review the [tutorial](https://aka.ms/aml-notebook-hyperdrive) on single-node TensorFlow training using the SDK

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.17


## Diagnostics
Opt-in diagnostics for better experience, quality, and security of future releases.

In [2]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


## Initialize workspace
Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [4]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Found the config file in: C:\Users\seretkow\notebooks\Labs Azure ML service\aml_config\config.json
Workspace name: MLServiceWorkspace
Azure region: westeurope
Resource group: mlserviceresourcegroup


## Create or Attach existing AmlCompute
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for training your model. In this tutorial, you create `AmlCompute` as your training compute resource.

**Creation of AmlCompute takes approximately 5 minutes.** If the AmlCompute with that name is already in your workspace this code will skip the creation process.

As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current cluster. 
print(compute_target.status.serialize())

Found existing compute target
{'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-02-22T14:28:25.671000+00:00', 'creationTime': '2018-11-23T16:53:14.040220+00:00', 'currentNodeCount': 0, 'errors': None, 'modifiedTime': '2018-11-23T16:53:56.988534+00:00', 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': ''}, 'targetNodeCount': None, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


The above code creates a GPU cluster. If you instead want to create a CPU cluster, provide a different VM size to the `vm_size` parameter, such as `STANDARD_D2_V2`.

## Upload data to datastore
To make data accessible for remote training, AML provides a convenient way to do so via a [Datastore](https://docs.microsoft.com/azure/machine-learning/service/how-to-access-data). The datastore provides a mechanism for you to upload/download data to Azure Storage, and interact with it from your remote compute targets. 

If your data is already stored in Azure, or you download the data as part of your training script, you will not need to do this step. For this tutorial, although you can download the data in your training script, we will demonstrate how to upload the training data to a datastore and access it during training to illustrate the datastore functionality.

First, download the training data from [here](http://mattmahoney.net/dc/text8.zip) to your local machine:

In [6]:
import os
import urllib

os.makedirs('./data', exist_ok=True)
download_url = 'http://mattmahoney.net/dc/text8.zip'
urllib.request.urlretrieve(download_url, filename='./data/text8.zip')

('./data/text8.zip', <http.client.HTTPMessage at 0x22189460588>)

Each workspace is associated with a default datastore. In this tutorial, we will upload the training data to this default datastore.

In [7]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

AzureFile mlserviceworks0764555666 azureml-filestore-f5bb1783-07f8-419d-a006-faaadbf5a5db


Upload the contents of the data directory to the path `./data` on the default datastore.

In [8]:
ds.upload(src_dir='data', target_path='data', overwrite=True, show_progress=True)

Uploading data\test-images.gz
Uploading data\test-labels.gz
Uploading data\text8.zip
Uploading data\train-images.gz
Uploading data\train-labels.gz
Uploaded data\test-labels.gz, 1 files out of an estimated total of 5
Uploaded data\train-labels.gz, 2 files out of an estimated total of 5
Uploaded data\test-images.gz, 3 files out of an estimated total of 5
Uploaded data\train-images.gz, 4 files out of an estimated total of 5
Uploaded data\text8.zip, 5 files out of an estimated total of 5


$AZUREML_DATAREFERENCE_42ceb934698f4aad9c87520d8959931c

For convenience, let's get a reference to the path on the datastore with the zip file of training data. We can do so using the `path` method. In the next section, we can then pass this reference to our training script's `--input_data` argument. 

In [9]:
path_on_datastore = 'data/text8.zip'
ds_data = ds.path(path_on_datastore)
print(ds_data)

$AZUREML_DATAREFERENCE_ac1a4884c1bf4d62b94e010b044409a0


## Train model on the remote compute

### Create a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on.

In [10]:
import os

project_folder = './tf-distr-hvd'
os.makedirs(project_folder, exist_ok=True)

Copy the training script `tf_horovod_word2vec.py` into this project directory.

In [11]:
import shutil

shutil.copy('tf_horovod_word2vec.py', project_folder)

'./tf-distr-hvd\\tf_horovod_word2vec.py'

### Create an experiment
Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this distributed TensorFlow tutorial. 

In [12]:
from azureml.core import Experiment

experiment_name = 'tf-distr-hvd'
experiment = Experiment(ws, name=experiment_name)

### Create a TensorFlow estimator
The AML SDK's TensorFlow estimator enables you to easily submit TensorFlow training jobs for both single-node and distributed runs. For more information on the TensorFlow estimator, refer [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-train-tensorflow).

In [13]:
from azureml.train.dnn import TensorFlow

script_params={
    '--input_data': ds_data
}

estimator= TensorFlow(source_directory=project_folder,
                      compute_target=compute_target,
                      script_params=script_params,
                      entry_script='tf_horovod_word2vec.py',
                      node_count=2,
                      process_count_per_node=1,
                      distributed_backend='mpi',
                      use_gpu=True)

The above code specifies that we will run our training script on `2` nodes, with one worker per node. In order to execute a distributed run using MPI/Horovod, you must provide the argument `distributed_backend='mpi'`. Using this estimator with these settings, TensorFlow, Horovod and their dependencies will be installed for you. However, if your script also uses other packages, make sure to install them via the `TensorFlow` constructor's `pip_packages` or `conda_packages` parameters.

Note that we passed our training data reference `ds_data` to our script's `--input_data` argument. This will 1) mount our datastore on the remote compute and 2) provide the path to the data zip file on our datastore.

### Submit job
Run your experiment by submitting your estimator object. Note that this call is asynchronous.

In [14]:
run = experiment.submit(estimator)
print(run)

Run(Experiment: tf-distr-hvd,
Id: tf-distr-hvd_1551699762_668e3b9c,
Type: azureml.scriptrun,
Status: Queued)


### Monitor your run
You can monitor the progress of the run with a Jupyter widget. Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [15]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

Alternatively, you can block until the script has completed training before running more code.

In [ ]:
run.wait_for_completion(show_output=True)